In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
from lfp_tools import (
    startup_local as startup,
    general,
    analysis
)
import utils.behavioral_utils as behavioral_utils
import utils.spike_utils as spike_utils
import os
import scipy.signal as ss
import utils.behavioral_utils as behavioral_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
SPECIES = 'nhp'
SUBJECT = 'SA'
EXP = "WCST"

PRE_INTERVAL = 0
POST_INTERVAL = 1500
INTERVAL_SIZE = 10
NUM_BINS_SMOOTH = 10
EVENT = "FeedbackOnset"

In [8]:
sess_name = "20180802"

In [9]:
bands = startup.get_bands(SPECIES, SUBJECT, EXP)
print(bands)

['0.5-nan', '4-5', '9-12', '13-17', '18-24', '26-34', '45-55', '65-87', '120-148']


In [13]:
band = bands[-1]

In [14]:
lfp_files = startup.get_bipole_notch_files(SPECIES, SUBJECT, EXP, sess_name)


In [21]:
behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
beh = pd.read_csv(behavior_path)
valid_beh = behavioral_utils.get_valid_trials(beh)
valid_beh["np_idx"] = np.arange(len(valid_beh))
t_start = valid_beh.FeedbackOnset.astype(int)
t_len = 2800
offset = -1300

/tmp/ipykernel_251877/775705022.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_beh["np_idx"] = np.arange(len(valid_beh))


In [16]:
#Grab xyz locations and brain areas of units
cl = startup.get_bipole_info(SPECIES, SUBJECT, EXP, sess_name)
xyz_locs = cl[['x','y','z']].values
#Gathering spike counts in interval
sig = np.empty((len(lfp_files), t_start.shape[0], t_len))
for i,f in enumerate(lfp_files):
    print('Load Files', i+1, '/', len(lfp_files))
    lfp = general.open_local_h5py_file(f)
    print('Done loading file, processing')
    if band.split('-')[1]=='nan':
        lfp_h = analysis.butter_pass_filter(lfp, float(band.split('-')[0]), 1000, 'high')
        lfp_a = np.abs(ss.hilbert(lfp_h))
        sig[i] = analysis.time_Slicer(lfp_a, t_start+offset, t_len)
    else:
        lfp_h = analysis.butter_pass_filter(lfp, int(band.split('-')[0]), 1000, 'high')
        lfp_b = analysis.butter_pass_filter(lfp_h, int(band.split('-')[1]), 1000, 'low')
        lfp_a = np.abs(ss.hilbert(lfp_b))
        sig[i] = analysis.time_Slicer(lfp_a, t_start+offset, t_len)


Load Files 1 / 80
Done loading file, processing


/src/lfp_tools/lfp_tools/analysis.py:2046: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  idx = timePoints[:,np.newaxis] + np.arange(0,timeLength)


Load Files 2 / 80
Done loading file, processing
Load Files 3 / 80
Done loading file, processing
Load Files 4 / 80
Done loading file, processing
Load Files 5 / 80
Done loading file, processing
Load Files 6 / 80
Done loading file, processing
Load Files 7 / 80
Done loading file, processing
Load Files 8 / 80
Done loading file, processing
Load Files 9 / 80
Done loading file, processing
Load Files 10 / 80
Done loading file, processing
Load Files 11 / 80
Done loading file, processing
Load Files 12 / 80
Done loading file, processing
Load Files 13 / 80
Done loading file, processing
Load Files 14 / 80
Done loading file, processing
Load Files 15 / 80
Done loading file, processing
Load Files 16 / 80
Done loading file, processing
Load Files 17 / 80
Done loading file, processing
Load Files 18 / 80
Done loading file, processing
Load Files 19 / 80
Done loading file, processing
Load Files 20 / 80
Done loading file, processing
Load Files 21 / 80
Done loading file, processing
Load Files 22 / 80
Done load

In [26]:
df = startup.get_behavior(SPECIES, SUBJECT, EXP, sess_name, import_obj_features=False)
df_sub = df[
    (df['response'].isin([200,206])) & \
    (df['badGroup']==0) & \
    (df['ignore']==0) & \
    (df['badTrials']==0) & \
    (df['group']>1) & \
    (df['group']<np.max(df.group.values))
]

In [28]:
good_trials = df_sub.trial

In [31]:
valid_beh

,TrialNumber,BlockNumber,TrialAfterRuleChange,TaskInterrupt,ConditionNumber,Response,ItemChosen,TrialType,CurrentRule,LastRule,...,Item1Shape,Item1Color,Item1Pattern,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern,np_idx
35,35,2,0,NaN,641,Incorrect,3.0,8,CIRCLE,CYAN,...,TRIANGLE,MAGENTA,POLKADOT,STAR,GREEN,SWIRL,SQUARE,YELLOW,ESCHER,0
36,36,2,1,NaN,627,Incorrect,3.0,8,CIRCLE,CYAN,...,TRIANGLE,YELLOW,ESCHER,STAR,GREEN,SWIRL,SQUARE,MAGENTA,POLKADOT,1
37,37,2,2,NaN,808,Incorrect,3.0,8,CIRCLE,CYAN,...,TRIANGLE,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL,2
38,38,2,3,NaN,783,Incorrect,1.0,8,CIRCLE,CYAN,...,TRIANGLE,CYAN,ESCHER,STAR,GREEN,SWIRL,SQUARE,YELLOW,POLKADOT,3
39,39,2,4,NaN,1116,Incorrect,1.0,8,CIRCLE,CYAN,...,TRIANGLE,CYAN,POLKADOT,STAR,MAGENTA,RIPPLE,SQUARE,YELLOW,SWIRL,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,1738,54,31,NaN,4072,Correct,0.0,7,GREEN,CYAN,...,SQUARE,CYAN,ESCHER,STAR,MAGENTA,POLKADOT,CIRCLE,YELLOW,SWIRL,1702
1739,1739,54,32,NaN,4429,Correct,0.0,7,GREEN,CYAN,...,SQUARE,CYAN,ESCHER,CIRCLE,MAGENTA,RIPPLE,TRIANGLE,YELLOW,POLKADOT,1703
1740,1740,54,33,NaN,4457,Correct,0.0,7,GREEN,CYAN,...,TRIANGLE,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE,1704
1741,1741,54,34,NaN,4380,Correct,0.0,7,GREEN,CYAN,...,CIRCLE,CYAN,POLKADOT,TRIANGLE,MAGENTA,ESCHER,SQUARE,YELLOW,SWIRL,1705


In [30]:
valid_beh[valid_beh.TrialNumber.isin(good_trials)]

,TrialNumber,BlockNumber,TrialAfterRuleChange,TaskInterrupt,ConditionNumber,Response,ItemChosen,TrialType,CurrentRule,LastRule,...,Item1Shape,Item1Color,Item1Pattern,Item2Shape,Item2Color,Item2Pattern,Item3Shape,Item3Color,Item3Pattern,np_idx
36,36,2,1,NaN,627,Incorrect,3.0,8,CIRCLE,CYAN,...,TRIANGLE,YELLOW,ESCHER,STAR,GREEN,SWIRL,SQUARE,MAGENTA,POLKADOT,1
37,37,2,2,NaN,808,Incorrect,3.0,8,CIRCLE,CYAN,...,TRIANGLE,YELLOW,RIPPLE,STAR,GREEN,POLKADOT,SQUARE,CYAN,SWIRL,2
46,46,2,11,NaN,812,Incorrect,2.0,8,CIRCLE,CYAN,...,TRIANGLE,YELLOW,SWIRL,STAR,CYAN,POLKADOT,SQUARE,GREEN,RIPPLE,11
47,47,2,12,NaN,908,Incorrect,3.0,8,CIRCLE,CYAN,...,TRIANGLE,GREEN,SWIRL,STAR,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,12
50,50,2,15,NaN,995,Correct,0.0,8,CIRCLE,CYAN,...,TRIANGLE,MAGENTA,POLKADOT,STAR,CYAN,ESCHER,SQUARE,GREEN,RIPPLE,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,1738,54,31,NaN,4072,Correct,0.0,7,GREEN,CYAN,...,SQUARE,CYAN,ESCHER,STAR,MAGENTA,POLKADOT,CIRCLE,YELLOW,SWIRL,1702
1739,1739,54,32,NaN,4429,Correct,0.0,7,GREEN,CYAN,...,SQUARE,CYAN,ESCHER,CIRCLE,MAGENTA,RIPPLE,TRIANGLE,YELLOW,POLKADOT,1703
1740,1740,54,33,NaN,4457,Correct,0.0,7,GREEN,CYAN,...,TRIANGLE,CYAN,POLKADOT,SQUARE,MAGENTA,ESCHER,CIRCLE,YELLOW,RIPPLE,1704
1741,1741,54,34,NaN,4380,Correct,0.0,7,GREEN,CYAN,...,CIRCLE,CYAN,POLKADOT,TRIANGLE,MAGENTA,ESCHER,SQUARE,YELLOW,SWIRL,1705


In [17]:
sig_avg = np.mean(np.reshape(sig, (sig.shape[0], sig.shape[1], -1, 100)), axis=3)

In [22]:
flattened = sig_avg.reshape(-1, 1)
iterables = [np.arange(sig_avg.shape[0]), np.arange(sig_avg.shape[1]), np.arange(sig_avg.shape[2])]
index = pd.MultiIndex.from_product(iterables, names=["UnitID", "np_idx", "TimeBinID"])
df = pd.DataFrame(flattened, index=index)
df = df.reset_index()
df = df.rename(columns={0: "Value"})
df["TimeBins"] = df.TimeBinID / 10


In [24]:
merged = pd.merge(df, valid_beh, on="np_idx")[["UnitID", "TimeBins", "Value", "TrialNumber"]]

In [25]:
len(merged)

3823680

### Check session firing rates generated

In [8]:
band = '120-148'

In [10]:
SESS_LFP_PATH = "/data/patrick_scratch/multi_sess/{sess_name}/{sess_name}_{band}_lfp_{pre_interval}_{event}_{post_interval}_{interval_size}_bins.pickle"
sess_name = "20180709"

In [11]:
lfp_path = SESS_LFP_PATH.format(
    sess_name=sess_name, 
    band=band,
    pre_interval=0,  # naming wrong, actually supposed to be 1300
    event=EVENT, 
    post_interval=POST_INTERVAL, 
    interval_size=10  # also named wrong, supposed to be 100
)
lfps = pd.read_pickle(lfp_path)

In [14]:
lfps_time = lfps[np.isclose(lfps.TimeBins, 0.0)]

In [16]:
lfps_trial = lfps_time[lfps_time.TrialNumber == 0]

In [20]:
lfps_time.groupby(["UnitID", "TrialNumber"]).ngroups

71040